In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
!pip install keras

In [ ]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK

!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

datFile =  "/content/shape_predictor_68_face_landmarks.dat"

--2024-01-25 12:30:24--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  15.1MB/s    in 5.2s    

2024-01-25 12:30:30 (11.7 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
from google.colab import drive, files
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to the zip file on Google Drive
zip_file_path = "/content/drive/MyDrive/Datas.zip"

# Specify the target folder for extraction in Colab
extracted_folder = "Ims"

# Create the target directory if it doesn't exist
os.makedirs(extracted_folder, exist_ok=True)

# Copy the zip file from Google Drive to Colab
!cp "{zip_file_path}" .

# Unzip the file
with zipfile.ZipFile("/content/drive/MyDrive/Datas.zip", "r") as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"File '{zip_file_path}' has been extracted to '{extracted_folder}'.")


Mounted at /content/drive
File '/content/drive/MyDrive/Datas.zip' has been extracted to 'Ims'.


In [ ]:
import matplotlib.pyplot as plt
import os

# Draw bounding boxes and keypoints on an image
def draw_facebox_and_keypoints(filename, result_list, save_path=None):
    # Load the image
    data = plt.imread(filename)
    # Plot the image
    plt.imshow(data)
    # Get the context for drawing boxes
    ax = plt.gca()
    # Plot each box
    for result in result_list:
        # Check if 'box' is present in the result dictionary
        if 'box' in result:
            # Get coordinates
            x, y, width, height = result['box']
            # Create the shape
            rect = plt.Rectangle((x, y), width, height, fill=False, color='orange')
            # Draw the box
            ax.add_patch(rect)

        # Check if 'keypoints' is present in the result dictionary
        if 'keypoints' in result:
            # Draw the dots
            for key, value in result['keypoints'].items():
                # Create and draw dot
                dot = plt.Circle(value, radius=2, color='red')
                ax.add_patch(dot)

    # Save or display the plot
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()
    plt.close()


In [ ]:
import dlib
import cv2
import os
import pandas as pd

# Initialize face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(datFile)  # You need to download this file

# Directory containing your images
images_directory = '/content/Ims/Camera Roll'
# Output directory to save resized images
resized_directory = 'resized_images'
os.makedirs(resized_directory, exist_ok=True)
target_height, target_width, channels = 256, 256, 3
for filename in os.listdir(images_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
      original_image_path = os.path.join(images_directory, filename)

        # Read the original image
      image = cv2.imread(original_image_path)
      resized_image = cv2.resize(image, (target_width, target_height))
      resized_image = np.array(resized_image)
      resized_path = os.path.join(resized_directory, f'resized_{filename}')
      cv2.imwrite(resized_path, resized_image)

In [ ]:
images_directory = '/content/resized_images'

# Output directory to save annotated images
output_directory = 'annotated_images'
os.makedirs(output_directory, exist_ok=True)

# Lists to store image paths and corresponding landmarks
image_paths = []
landmarks_list = []


# Iterate through images in the directory
for filename in os.listdir(images_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        image_path = os.path.join(images_directory, filename)
        image = cv2.imread(image_path)



        # Detect faces in the image
        faces = detector(image)

        # Loop through detected faces
        for face_index, face in enumerate(faces):
            # Get bounding box coordinates
            x, y, w, h = face.left(), face.top(), face.width(), face.height()

            # Get facial landmarks
            landmarks = predictor(image, face)
            keypoints = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]  # Assuming 68 landmarks

            # Save or display the annotated image for each face
            output_path = os.path.join(output_directory, f'annotated_{filename[:-4]}_face{face_index}.png')
            draw_facebox_and_keypoints(image_path, [{'box': (x, y, w, h), 'keypoints': dict(enumerate(keypoints))}], save_path=output_path)

            # Append image path and landmarks to the lists
            image_paths.append(image_path)
            landmarks_list.append(keypoints)

# Create a DataFrame to store image paths and landmarks
df = pd.DataFrame({'image_path': image_paths, 'landmarks': landmarks_list})

# Save the DataFrame to a CSV file (adjust the filename)
df.to_csv('landmarks_annotations.csv', index=False)

print("Annotation of images completed.")

Annotation of images completed.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, PReLU, MaxPooling2D, Flatten, Dense, Softmax, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence

# Load the CSV file containing image paths and landmarks
annotations_file = 'landmarks_annotations.csv'
annotations_df = pd.read_csv(annotations_file)

# Assuming landmarks are stored as a string in the DataFrame; convert them to numpy arrays
annotations_df['landmarks'] = annotations_df['landmarks'].apply(eval)

# Extract image paths and landmarks
image_paths = annotations_df['image_path'].values
landmarks = np.array(list(annotations_df['landmarks'].values))

# Load and preprocess images
images = []
import cv2

# # Assuming you have a target height, width, and channels
# target_height, target_width, channels = 128, 128, 3

for image_path in image_paths:
    image = cv2.imread(image_path)

#     # Resize the image to the target size
#     image = cv2.resize(image, (target_width, target_height))

    # Normalize pixel values to the range [0, 1]
    image = image / 255.0

#     # Add the preprocessed image to the list
    images.append(image)

# # Convert the list of images to a numpy array
# images = np.array(images)

# Flatten landmarks to match the output shape of the model
landmarks = landmarks.reshape((landmarks.shape[0], -1))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, landmarks, test_size=0.2, random_state=42)


# Create an instance of the model
def LandmarkDetector(input_shape=(256, 256, 3)):
    input_tensor = Input(shape=input_shape, name='data')

    x = Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation=None, name='conv1')(input_tensor)
    x = PReLU(name='prelu1')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', name='pool1')(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', activation=None, name='conv2')(x)
    x = PReLU(name='prelu2')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', name='pool2')(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', activation=None, name='conv3')(x)
    x = PReLU(name='prelu3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)

    x = Conv2D(128, (2, 2), strides=(1, 1), padding='valid', activation=None, name='conv4')(x)
    x = PReLU(name='prelu4')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(256, activation=None, name='conv5')(x)
    x = PReLU(name='prelu5')(x)

    # Adding dropout for regularization
    x = Dropout(0.5)(x)

    landmarks = Dense(136, activation=None, name='landmarks')(x)  # 68 points * 2 coordinates

    output1 = Softmax(name='prob1')(landmarks)

    model = Model(inputs=input_tensor, outputs=[output1, landmarks])

    return model

# Create an instance of the model
landmark_detector_model = LandmarkDetector()

# Compile the model
landmark_detector_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Display the model summary
landmark_detector_model.summary()

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Create a custom data generator
class LandmarkDataGenerator(Sequence):
    def __init__(self, X, y, batch_size, datagen):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.datagen = datagen

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size

        batch_X = self.X[start:end]
        batch_y = self.y[start:end]

        # Apply data augmentation
        augmented_images, augmented_landmarks = [], []
        for i in range(len(batch_X)):
            image = batch_X[i]
            landmark = batch_y[i]

            augmented_image, _ = self.datagen.flow(np.expand_dims(image, axis=0), np.zeros(1), batch_size=1).next()
            augmented_landmark = landmark  # No augmentation for landmarks, adjust as needed

            augmented_images.append(augmented_image.squeeze())
            augmented_landmarks.append(augmented_landmark)

        return np.array(augmented_images), [np.array(augmented_landmarks), np.array(augmented_landmarks)]

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create custom data generators
batch_size = 16  # Adjust the batch size as needed

train_data_generator = LandmarkDataGenerator(X_train, y_train, batch_size=batch_size, datagen=datagen)
val_data_generator = LandmarkDataGenerator(X_val, y_val, batch_size=batch_size, datagen=datagen)

# Calculate steps per epoch and validation steps
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

# If the number of samples is not perfectly divisible by the batch size, you can add an additional step
if len(X_train) % batch_size != 0:
    steps_per_epoch += 1

if len(X_val) % batch_size != 0:
    validation_steps += 1

# Train the model using the augmented data
history = landmark_detector_model.fit_generator(
    train_data_generator,
    epochs=20,
    validation_data=val_data_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

# Evaluate the model on the validation set
loss = landmark_detector_model.evaluate_generator(val_data_generator, steps=validation_steps)
print(f"Validation Loss: {loss}")


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 data (InputLayer)           [(None, 256, 256, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 254, 254, 32)      896       
                                                                 
 prelu1 (PReLU)              (None, 254, 254, 32)      2064512   
                                                                 
 pool1 (MaxPooling2D)        (None, 126, 126, 32)      0         
                                                                 
 conv2 (Conv2D)              (None, 124, 124, 64)      18496     
                                                                 
 prelu2 (PReLU)              (None, 124, 124, 64)      984064    
                                                                 
 pool2 (MaxPooling2D)        (None, 61, 61, 64)        0   

<ipython-input-10-203bec37e50d>:157: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = landmark_detector_model.fit_generator(


Epoch 1/20
15/15 [==============================] - 84s 4s/step - loss: 30483.2031 - prob1_loss: 19753.0840 - landmarks_loss: 10730.1211 - prob1_accuracy: 0.0000e+00 - landmarks_accuracy: 0.0000e+00 - val_loss: 21385.2754 - val_prob1_loss: 19939.6035 - val_landmarks_loss: 1445.6707 - val_prob1_accuracy: 0.0000e+00 - val_landmarks_accuracy: 0.0000e+00
Epoch 2/20
15/15 [==============================] - 59s 4s/step - loss: 22844.2559 - prob1_loss: 19752.8613 - landmarks_loss: 3091.3960 - prob1_accuracy: 0.0474 - landmarks_accuracy: 0.0474 - val_loss: 20335.5898 - val_prob1_loss: 19939.5547 - val_landmarks_loss: 396.0378 - val_prob1_accuracy: 0.1034 - val_landmarks_accuracy: 0.1034
Epoch 3/20
15/15 [==============================] - 59s 4s/step - loss: 21439.3477 - prob1_loss: 19752.7363 - landmarks_loss: 1686.6095 - prob1_accuracy: 0.0603 - landmarks_accuracy: 0.0603 - val_loss: 20297.6660 - val_prob1_loss: 19939.5156 - val_landmarks_loss: 358.1509 - val_prob1_accuracy: 0.4310 - val_land

<ipython-input-10-203bec37e50d>:166: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss = landmark_detector_model.evaluate_generator(val_data_generator, steps=validation_steps)


Validation Loss: [20063.111328125, 19939.505859375, 123.60707092285156, 0.43103447556495667, 0.43103447556495667]


In [ ]:
import matplotlib.pyplot as plt

# Function to preprocess an image
def preprocess_image(img):
    # Resize the image to the input size expected by your model
    target_height, target_width = 256, 256  # Adjust based on your model's input size
    img_resized = cv2.resize(img, (target_width, target_height))

    # Normalize pixel values to the range [0, 1]
    img_normalized = img_resized / 255.0

    return img_normalized
# Function to draw landmarks on the image using Matplotlib
def draw_landmarks_matplotlib(img, landmarks, save_path=None):
    # Assuming landmarks is a list with the second element containing coordinates
    landmark_coordinates = landmarks[1][0]

    # Reshape the flat array into pairs of (x, y) coordinates
    landmark_coordinates = landmark_coordinates.reshape((-1, 2))

    # Create a figure and axis
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # Draw circles at each landmark position
    for x, y in landmark_coordinates:
        circle = plt.Circle((x, y), radius=2, color='green')
        ax.add_patch(circle)

    # Save or display the plot
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close()



# Path to the single image you want to test
image_path = '/content/WIN_20240125_12_57_04_Pro.jpg'

# Read the image
img = cv2.imread(image_path)

# Preprocess the image
img_preprocessed = preprocess_image(img)
print(img_preprocessed.shape)
# Predict landmarks using your model
landmarks = landmark_detector_model.predict(np.expand_dims(img_preprocessed, axis=0))

# Draw landmarks on the image using Matplotlib
draw_landmarks_matplotlib(img_preprocessed, landmarks)


In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
import matplotlib.pyplot as plt

# Function to preprocess an image
def preprocess_image(img):
    # Resize the image to the input size expected by your model
    target_height, target_width = 256, 256  # Adjust based on your model's input size
    img_resized = cv2.resize(img, (target_width, target_height))

    # Normalize pixel values to the range [0, 1]
    img_normalized = img_resized / 255.0

    return img_normalized
# Function to draw landmarks on the image using Matplotlib
def draw_landmarks_matplotlib(img, landmarks, save_path=None):
    # Assuming landmarks is a list with the second element containing coordinates
    landmark_coordinates = landmarks[1][0]

    # Reshape the flat array into pairs of (x, y) coordinates
    landmark_coordinates = landmark_coordinates.reshape((-1, 2))

    # Create a figure and axis
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # Draw circles at each landmark position
    for x, y in landmark_coordinates:
        circle = plt.Circle((x, y), radius=2, color='green')
        ax.add_patch(circle)

    # Save or display the plot
    if save_path:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close()



# Path to the single image you want to test
image_path = '/content/WIN_20240125_12_57_04_Pro.jpg'

# Read the image
img = cv2.imread(image_path)

# Preprocess the image
img_preprocessed = preprocess_image(img)
# print(img_preprocessed.shape)
# Predict landmarks using your model
landmarks = landmark_detector_model.predict(np.expand_dims(img_preprocessed, axis=0))

# Draw landmarks on the image using Matplotlib
draw_landmarks_matplotlib(img_preprocessed, landmarks)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialize bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])


    img_preprocessed = preprocess_image(img)
# print(img_preprocessed.shape)
# Predict landmarks using your model
    landmarks = landmark_detector_model.predict(np.expand_dims(img_preprocessed, axis=0))

# Draw landmarks on the image using Matplotlib
    draw_landmarks_matplotlib(img_preprocessed, landmarks)


    # save the image with landmarks
    cv2.imwrite(f'frame_{count}.png', cv2.cvtColor((img * 255.0).astype(np.uint8), cv2.COLOR_RGB2BGR))

    count += 1
